In [ ]:
sub accuracy_metric{
    my ($self, $actual, $predicted) = @_;
    my $cmp = $predicted->astype($actual->dtype) == $actual;
    return sprintf '%0.2f',
                       (100 * $cmp->astype($actual->dtype)->sum / $actual->len)->asscalar;
}
sml->add_to_class('accuracy_metric', \&{'accuracy_metric'});

In [ ]:




sub confusion_matrix{
    my ($self, $actual, $predicted) = @_;
    
    #Step 1: One-hot encode the actual and predicted arrays
    my $num_classes       = $actual->max->asscalar + 1;
    my $actual_one_hot    = mx->nd->one_hot($actual, $num_classes);
    my $predicted_one_hot = mx->nd->one_hot($predicted, $num_classes);
                                            
    # Step 2: Compute confusion matrix
    # Matrix multiplication: (actual_one_hot^T) * predicted_one_hot
    return mx->nd->arange(stop=>$num_classes),
           mx->nd->dot($actual_one_hot->T, $predicted_one_hot); 
}

sml->add_to_class('confusion_matrix', \&{'confusion_matrix'});

In [ ]:
# Function To Pretty Print a Confusion Matrix.
# pretty print a confusion matrix
sub print_confusion_matrix{
    my ($self, $unique, $matrix) = @_;
    printf "A/P%s", $unique->aspdl;
    printf "%s", mx->nd->concat($unique->expand_dims(axis=>1), $matrix, dim=>1)->aspdl;
}

sml->add_to_class('printf_confusion_matrix', \&{'print_confusion_matrix'});

In [ ]:
# Function To Calculate Mean Absolute Error.
# Calculate mean absolute error
sub mae_metric{
    my ($self, $actual, $predicted) = @_;
    return sprintf '%0.2f',
                    (mx->nd->abs($actual - $predicted)->sum / $actual->len)->asscalar;       
}

sml->add_to_class('mae_metric', \&{'mae_metric'});

In [ ]:
# Defined in Section 4.2.4 Root Mean Squared Error
# Function To Calculate Root Mean Squared Error.
# Calculate root mean squared error
sub rmse_metric{
    my($self, $actual, $predicted) = @_;
    my $mean_error = ($actual - $predicted)->square()->sum / $actual->len;
    return sprintf '%0.3f', $mean_error->sqrt()->asscalar;
}

sml->add_to_class('rmse_metric', \&{'rmse_metric'});

In [ ]:
# Function to calculate the ROC metrics by using one-hot encoding and dot product
sub perf_metrics{
    my ($self, $actual, $predicted_prob, $threshold) = @_;
        
    my ($tp, $fp, $tn, $fn, $tpr, $fpr) = (0, 0, 0, 0);
        
    #Step 1: Threshold to create binary predictions
    my $predicted = $predicted_prob >= $threshold;

    #Step 2: Convert actua and predicted to one-hot encoded matrices
    my $num_classes         = $actual->max->asscalar + 1;
    my $actual_one_hot      = mx->nd->one_hot($actual, $num_classes); #Shape [n, $num_cl]
    my $predicted_one_hot   = mx->nd->one_hot($predicted, $num_classes); #Shape [n, $num_cl]
    
    #Step 3: Compute confusion matrix using dot product
    my $confusion_matrix    = mx->nd->dot($actual_one_hot->T, $predicted_one_hot);
    
    #Extract counts from the confusion matrix
    $tp = $confusion_matrix->at(1)->at(1); # True Positives
    $fn = $confusion_matrix->at(1)->at(0); # False Negatives
    $fp = $confusion_matrix->at(0)->at(1); # False Positives
    $tn = $confusion_matrix->at(0)->at(0); # True Negatives

    #Step 4: Compute TPR and FPR
    $tpr = $tp / ($tp + $fn); # True Positive Rate
    $fpr = $fp / ($fp + $tn); # False Positive Rate

    return sprintf('%0.2f', $fpr->asscalar), sprintf('%0.2f', $tpr->asscalar);
}

sml->add_to_class('perf_metrics', \&{'perf_metrics'});